#### 1. Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. (У aviasales есть для этого дополнительное API) Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)

In [1]:
import pandas as pd
import requests
import json
from pprint import pprint

In [2]:
# %load aviasales.py
from pprint import pprint
import requests
import json
service = 'http://min-prices.aviasales.ru/calendar_preload?'
fromCity = 'MOW'
toCity = 'LED'
link = f'{service}origin={fromCity}&destination={toCity}&one_way=true'
req = requests.get(link)
data = json.loads(req.text)
for i in data['best_prices']:
    print(i['value'],i['depart_date'],i['return_date'],i['gate'])


2200.0 2019-09-09 None Pobeda
2200.0 2019-11-16 None Pobeda
3815.0 2020-05-09 None S7
2200.0 2019-09-24 None Pobeda
2490.0 2019-09-19 None Aviakassa
3032.0 2019-09-13 None OneTwoTrip
2490.0 2019-10-18 None Aviakassa
2200.0 2019-09-17 None Pobeda
3100.0 2019-12-29 None Pobeda
3100.0 2019-08-20 None Pobeda
1499.0 2020-03-05 None Pobeda
3815.0 2020-07-18 None S7
14315.0 2020-08-04 None Aerobilet
3815.0 2020-06-21 None S7
3815.0 2020-06-03 None S7
3100.0 2019-08-22 None Pobeda
1795.0 2020-02-28 None SuperKassa.ru
3815.0 2020-06-23 None S7
2200.0 2019-09-18 None Pobeda
3815.0 2020-04-30 None S7
1499.0 2019-12-06 None Pobeda
1499.0 2020-02-27 None Pobeda
2618.0 2019-09-07 None OneTwoTrip
3815.0 2020-05-02 None OneTwoTrip
1795.0 2020-02-01 None SuperKassa.ru
3815.0 2020-06-30 None S7
1795.0 2020-01-18 None SuperKassa.ru
3815.0 2020-05-20 None OneTwoTrip
1499.0 2020-02-09 None Pobeda
2200.0 2020-01-04 None Pobeda
1795.0 2019-11-23 None SuperKassa.ru
3815.0 2020-04-16 None S7
2143.0 2019-10-02 

In [3]:
def get_cities():
    
    link = 'http://api.travelpayouts.com/data/ru/cities.json'
    
    with requests.session() as s:
        r = s.get(link)
        
    data = json.loads(r.text)
    
    cities = dict()
    
    for item in data:
        cities[item['name']] = item['code']
        cities[item['name_translations']['en']] = item['code']
        
    return cities

    
def get_tickets(orig_name, dest_name, one_way_ticket=True, mode='p'):
    
    """
    p - print data
    f - get DataFrame with data
    """
    
    cities = get_cities()

    origin = cities.get(orig_name, '--')
    destination = cities.get(dest_name, '--')

    if one_way_ticket:
        one_way = 'true'
    else:
        one_way = 'false'

    service = 'http://min-prices.aviasales.ru/calendar_preload?'

    link = f'{service}origin={origin}&destination={destination}&one_way={one_way}'

    with requests.session() as s:
        r = s.get(link)

    data = json.loads(r.text)

    tickets = {
        'sequence_number': [],
        'origin': [],
        'destination': [],
        'gate': [],
        'depart_date': [],
        'return_date': [],
        'number_of_changes': [],
        'value': []
    }

    for idx, item in enumerate(data['best_prices']):
        tickets['sequence_number'].append(idx+1)
        tickets['origin'].append(orig_name)
        tickets['destination'].append(dest_name)
        gate = item['gate']
        tickets['gate'].append(gate)
        depart_date = item['depart_date']
        tickets['depart_date'].append(depart_date)
        return_date = item['return_date']
        tickets['return_date'].append(return_date)
        number_of_changes = item['number_of_changes']
        tickets['number_of_changes'].append(number_of_changes)
        value = item['value']
        tickets['value'].append(item['value'])
        
        if mode == 'p':
            print('*' * 42)
            print(f'Вариант №:             {idx+1}')
            print(f'Пункт отправления:     {orig_name}')
            print(f'Пункт назначения:      {dest_name}')
            print(f'Продавец:              {dest_name}')
            print(f'Дата отправления:      {depart_date}')
            print(f'Дата возвращения:      {return_date}')
            print(f'Количество пересадок:  {number_of_changes}')
            print(f'Цена билета:           {value}')
            print('*' * 42)
            print('\n')
            
    if mode == 'f':
        return pd.DataFrame(tickets).sort_values(by=['depart_date', 'number_of_changes', 'value']).reset_index(drop=True)
    else:
        return


In [4]:
get_tickets('Москва', 'Волгоград')

******************************************
Вариант №:             1
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Волгоград
Дата отправления:      2020-01-06
Дата возвращения:      None
Количество пересадок:  0
Цена билета:           3100.0
******************************************


******************************************
Вариант №:             2
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Волгоград
Дата отправления:      2019-11-16
Дата возвращения:      None
Количество пересадок:  0
Цена билета:           2300.0
******************************************


******************************************
Вариант №:             3
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Волгоград
Дата отправления:      2019-10-10
Дата возвращения:      None
Количество пересадок:  0
Цена билета:           2700.0
******************************************


**********************

Количество пересадок:  0
Цена билета:           2700.0
******************************************


******************************************
Вариант №:             157
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Волгоград
Дата отправления:      2020-01-03
Дата возвращения:      None
Количество пересадок:  0
Цена билета:           3100.0
******************************************


******************************************
Вариант №:             158
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Волгоград
Дата отправления:      2019-08-13
Дата возвращения:      None
Количество пересадок:  0
Цена билета:           3907.0
******************************************


******************************************
Вариант №:             159
Пункт отправления:     Москва
Пункт назначения:      Волгоград
Продавец:              Волгоград
Дата отправления:      2019-11-26
Дата возвращения:      None
Количество перес

In [5]:
df = get_tickets('Москва', 'Волгоград', mode='f')
df.head(10)

,sequence_number,origin,destination,gate,depart_date,return_date,number_of_changes,value
0,162,Москва,Волгоград,KupiBilet.ru,2019-08-08,None,2,10378.0
1,32,Москва,Волгоград,KupiBilet.ru,2019-08-09,None,1,4752.0
2,111,Москва,Волгоград,Pobeda,2019-08-10,None,0,4100.0
3,104,Москва,Волгоград,S7,2019-08-11,None,0,4804.0
4,40,Москва,Волгоград,Pobeda,2019-08-12,None,0,4100.0
5,31,Москва,Волгоград,Nordwind,2019-08-13,None,0,3907.0
6,143,Москва,Волгоград,Nordwind,2019-08-14,None,0,3907.0
7,99,Москва,Волгоград,Pobeda,2019-08-15,None,0,4100.0
8,207,Москва,Волгоград,KupiBilet.ru,2019-08-16,None,1,4551.0
9,108,Москва,Волгоград,Pobeda,2019-08-17,None,0,4100.0


#### 2. В приложении парсинга википедии получить ссылки на соседние страницы и вывести все значимые слова из них

In [6]:
import re

In [7]:
# %load wiki.py
from pprint import pprint
import requests
import re

def get_link(topic):
    link='https://ru.wikipedia.org/wiki/'+topic.capitalize()
    return link

def get_topic_page(topic):
    link = get_link(topic)
    html = requests.get(link).text
    return html

def get_topic_text(topic):
    html_content = get_topic_page(topic)
    words = re.findall("[а-яА-Я]{3,}",html_content)
    #text = ' '.join(words)
    return words

# text = get_topic_text('Дерево')
# print(len(text))
# print(text[0:1000])

def get_common_words(topic):
    words_list = get_topic_text(topic)
    rate={}
    for word in words_list:
        if word in rate:
            rate[word]+=1
        else:
            rate[word]=1
    rate_list = list(rate.items())
    rate_list.sort(key = lambda x: -x[1])
    return rate_list

dict1 = get_common_words('Россия')
pprint(dict1)


[('России', 693),
 ('года', 242),
 ('Российской', 229),
 ('Федерации', 216),
 ('Россия', 211),
 ('отношения', 205),
 ('Российско', 162),
 ('году', 147),
 ('СССР', 128),
 ('страница', 101),
 ('Российская', 99),
 ('отсутствует', 98),
 ('языке', 96),
 ('Республика', 94),
 ('населения', 93),
 ('год', 84),
 ('также', 77),
 ('неопредел', 76),
 ('нном', 76),
 ('Википедия', 75),
 ('неопр', 74),
 ('мире', 67),
 ('стран', 67),
 ('как', 64),
 ('мира', 60),
 ('Москва', 60),
 ('для', 58),
 ('статья', 58),
 ('Александр', 58),
 ('место', 56),
 ('страны', 56),
 ('Дата', 55),
 ('государство', 54),
 ('государства', 54),
 ('что', 54),
 ('Увеличить', 54),
 ('млн', 54),
 ('тыс', 53),
 ('обращения', 53),
 ('Статьи', 51),
 ('Список', 51),
 ('является', 49),
 ('государственной', 49),
 ('СНГ', 49),
 ('История', 48),
 ('ООН', 47),
 ('Основная', 47),
 ('Архивировано', 47),
 ('Совет', 44),
 ('века', 44),
 ('Санкт', 43),
 ('лет', 43),
 ('января', 42),
 ('Владимир', 42),
 ('около', 42),
 ('служба', 41),
 ('Федераци

 ('Прикаспийская', 3),
 ('Горный', 3),
 ('хребет', 3),
 ('рек', 3),
 ('Длина', 3),
 ('Площадь', 3),
 ('Нижняя', 3),
 ('Башкортостане', 3),
 ('газ', 3),
 ('Южнее', 3),
 ('обычно', 3),
 ('подзолистые', 3),
 ('Западной', 3),
 ('много', 3),
 ('пахотных', 3),
 ('зональность', 3),
 ('полный', 3),
 ('зон', 3),
 ('арктических', 3),
 ('климат', 3),
 ('низких', 3),
 ('растительность', 3),
 ('животный', 3),
 ('существует', 3),
 ('внутри', 3),
 ('Они', 3),
 ('типы', 3),
 ('ландшафты', 3),
 ('Тундра', 3),
 ('тундра', 3),
 ('Широколиственный', 3),
 ('Степь', 3),
 ('физико', 3),
 ('различаются', 3),
 ('автономный', 3),
 ('Ярославль', 3),
 ('относятся', 3),
 ('хвойно', 3),
 ('широколиственных', 3),
 ('Волге', 3),
 ('значительная', 3),
 ('Провинции', 3),
 ('Ирано', 3),
 ('туранская', 3),
 ('определяют', 3),
 ('орех', 3),
 ('рная', 3),
 ('Красная', 3),
 ('книгу', 3),
 ('птиц', 3),
 ('Белый', 3),
 ('северный', 3),
 ('вка', 3),
 ('виды', 3),
 ('регионе', 3),
 ('тур', 3),
 ('Столицы', 3),
 ('середина', 3),

 ('Воеводский', 2),
 ('Окуньков', 2),
 ('Смирнов', 2),
 ('престижной', 2),
 ('наградой', 2),
 ('Пинскер', 2),
 ('Холево', 2),
 ('Премия', 2),
 ('Шеннона', 2),
 ('Громов', 2),
 ('математике', 2),
 ('Пуанкаре', 2),
 ('планирует', 2),
 ('уволить', 2),
 ('Развитие', 2),
 ('журналов', 2),
 ('большим', 2),
 ('программированию', 2),
 ('патентных', 2),
 ('заявок', 2),
 ('восходит', 2),
 ('Его', 2),
 ('Глушко', 2),
 ('ранних', 2),
 ('искусственный', 2),
 ('спутник', 2),
 ('Луноход', 2),
 ('космическая', 2),
 ('Салют', 2),
 ('космического', 2),
 ('Буран', 2),
 ('ракетно', 2),
 ('Спейс', 2),
 ('шаттл', 2),
 ('единственным', 2),
 ('планируется', 2),
 ('корабль', 2),
 ('экипажем', 2),
 ('Кустодиев', 2),
 ('Купчиха', 2),
 ('чаем', 2),
 ('отличается', 2),
 ('высокой', 2),
 ('диалогу', 2),
 ('духовной', 2),
 ('культурное', 2),
 ('искусства', 2),
 ('современном', 2),
 ('Современник', 2),
 ('Дружинин', 2),
 ('нного', 2),
 ('составляющей', 2),
 ('законе', 2),
 ('благодати', 2),
 ('Память', 2),
 ('похвала

 ('туполистная', 1),
 ('фисташка', 1),
 ('Сумах', 1),
 ('сумах', 1),
 ('Земляничное', 1),
 ('земляничное', 1),
 ('Самшит', 1),
 ('вечнозел', 1),
 ('самшит', 1),
 ('Падуб', 1),
 ('падуб', 1),
 ('Витекс', 1),
 ('священный', 1),
 ('витекс', 1),
 ('Чубушник', 1),
 ('чубушник', 1),
 ('Миндаль', 1),
 ('миндаль', 1),
 ('Лавровишня', 1),
 ('лавровишня', 1),
 ('Клекачка', 1),
 ('колхидская', 1),
 ('клекачка', 1),
 ('Ладанник', 1),
 ('ладанник', 1),
 ('Плющ', 1),
 ('плющ', 1),
 ('Сассапариль', 1),
 ('сассапариль', 1),
 ('Обвойник', 1),
 ('обвойник', 1),
 ('Ломонос', 1),
 ('ломонос', 1),
 ('Хмель', 1),
 ('обыкновенный', 1),
 ('хмель', 1),
 ('парк', 1),
 ('заповедников', 1),
 ('заповедника', 1),
 ('увеличившаяся', 1),
 ('Обезлесение', 1),
 ('вырублено', 1),
 ('лесных', 1),
 ('массивов', 1),
 ('сократилась', 1),
 ('Распространена', 1),
 ('нелегальная', 1),
 ('Западе', 1),
 ('Дальнем', 1),
 ('Востоке', 1),
 ('теряет', 1),
 ('заброшенных', 1),
 ('зарастает', 1),
 ('Занес', 1),
 ('Красную', 1),
 ('мле

 ('Поправки', 1),
 ('поправок', 1),
 ('приняты', 1),
 ('Данные', 1),
 ('поправки', 1),
 ('увеличившие', 1),
 ('международный', 1),
 ('следующие', 1),
 ('базовые', 1),
 ('принципы', 1),
 ('Народовластие', 1),
 ('народовластие', 1),
 ('Федерализм', 1),
 ('федерализм', 1),
 ('Разделение', 1),
 ('разделение', 1),
 ('приоритет', 1),
 ('Местное', 1),
 ('местное', 1),
 ('Многопартийность', 1),
 ('многопартийность', 1),
 ('идеологическое', 1),
 ('политическое', 1),
 ('Верховенство', 1),
 ('верховенство', 1),
 ('форма', 1),
 ('публично', 1),
 ('Субсидиарность', 1),
 ('субсидиарный', 1),
 ('закреплены', 1),
 ('необходима', 1),
 ('детальная', 1),
 ('проработка', 1),
 ('совершенствование', 1),
 ('Правовое', 1),
 ('механизма', 1),
 ('принципами', 1),
 ('устанавливаемыми', 1),
 ('законодательный', 1),
 ('представительный', 1),
 ('Парламенты', 1),
 ('законодательное', 1),
 ('должностного', 1),
 ('Действующие', 1),
 ('губернатор', 1),
 ('замещать', 1),
 ('указанную', 1),
 ('полномочный', 1),
 ('террит

 ('спадом', 1),
 ('уменьшается', 1),
 ('дневных', 1),
 ('отделениях', 1),
 ('Учреждения', 1),
 ('законодательством', 1),
 ('проходить', 1),
 ('периодическую', 1),
 ('аттестацию', 1),
 ('лицензирование', 1),
 ('государственную', 1),
 ('аккредитацию', 1),
 ('Учебные', 1),
 ('аккредитации', 1),
 ('выдавать', 1),
 ('документы', 1),
 ('дипломы', 1),
 ('свидетельства', 1),
 ('вообще', 1),
 ('вести', 1),
 ('образовательную', 1),
 ('Понятие', 1),
 ('аттестации', 1),
 ('учреждениям', 1),
 ('образовательным', 1),
 ('выпускникам', 1),
 ('производится', 1),
 ('массовый', 1),
 ('двухуровневую', 1),
 ('Бакалавр', 1),
 ('бакалавр', 1),
 ('Магистр', 1),
 ('магистр', 1),
 ('Одноуровневая', 1),
 ('получением', 1),
 ('квалификации', 1),
 ('Специалист', 1),
 ('квалификация', 1),
 ('профессий', 1),
 ('имеющих', 1),
 ('оборонное', 1),
 ('бесплатных', 1),
 ('бюджетных', 1),
 ('обучения', 1),
 ('платы', 1),
 ('снижает', 1),
 ('общую', 1),
 ('комбинированный', 1),
 ('рассчитываемый', 1),
 ('совокупной', 1),
 (

 ('Лесные', 1),
 ('Беларуси', 1),
 ('показателях', 1),
 ('устойчивого', 1),
 ('Вырубка', 1),
 ('экологических', 1),
 ('Биологические', 1),
 ('Позвоночные', 1),
 ('животные', 1),
 ('Другие', 1),
 ('заявлении', 1),
 ('закреплением', 1),
 ('украинского', 1),
 ('единственного', 1),
 ('соотечественников', 1),
 ('Этногенез', 1),
 ('Румянцев', 1),
 ('славянского', 1),
 ('народности', 1),
 ('Седова', 1),
 ('Русин', 1),
 ('Кишин', 1),
 ('Фомин', 1),
 ('Варяго', 1),
 ('Кочетков', 1),
 ('Эволюция', 1),
 ('Коломна', 1),
 ('Тимонин', 1),
 ('генезис', 1),
 ('Историко', 1),
 ('методологический', 1),
 ('автореферат', 1),
 ('Потапов', 1),
 ('Реконструкция', 1),
 ('Два', 1),
 ('этапа', 1),
 ('Археолог', 1),
 ('нститут', 1),
 ('археолог', 1),
 ('Беляев', 1),
 ('взаимоотношений', 1),
 ('Причерноморья', 1),
 ('Руденко', 1),
 ('православного', 1),
 ('христианства', 1),
 ('дорова', 1),
 ('Правонарушение', 1),
 ('юридическая', 1),
 ('правде', 1),
 ('Тольятти', 1),
 ('Маркелов', 1),
 ('Возникновение', 1),
 ('р

 ('Мадагаскар', 1),
 ('малавийские', 1),
 ('Малави', 1),
 ('малийские', 1),
 ('марокканские', 1),
 ('мозамбикские', 1),
 ('Намибийско', 1),
 ('Намибия', 1),
 ('нигерские', 1),
 ('нигерийские', 1),
 ('руандские', 1),
 ('Руанда', 1),
 ('свазилендские', 1),
 ('Свазиленд', 1),
 ('сейшельские', 1),
 ('Сейшелы', 1),
 ('сенегальские', 1),
 ('сомалийские', 1),
 ('суданские', 1),
 ('сьерра', 1),
 ('леонские', 1),
 ('танзанийские', 1),
 ('Танзания', 1),
 ('тоголезские', 1),
 ('тунисские', 1),
 ('угандские', 1),
 ('центральноафриканские', 1),
 ('ЦАР', 1),
 ('чадские', 1),
 ('экваториальногвинейские', 1),
 ('эритрейские', 1),
 ('Эритрея', 1),
 ('эфиопские', 1),
 ('Эфиопия', 1),
 ('южносуданские', 1),
 ('антигуанские', 1),
 ('Антигуа', 1),
 ('Барбуда', 1),
 ('аргентинские', 1),
 ('багамские', 1),
 ('Багамы', 1),
 ('барбадосские', 1),
 ('Барбадос', 1),
 ('белизские', 1),
 ('Белиз', 1),
 ('боливийские', 1),
 ('Боливия', 1),
 ('венесуэльские', 1),
 ('Венесуэла', 1),
 ('гаитянские', 1),
 ('Гаити', 1),


In [8]:
def get_linked_page(topic):
    html_content = get_topic_page(topic)
    links = [link[0] for link in re.findall('"((http)s?://.*?)"',html_content)]
    html_list = []
    for link in links[:20]: # ограничение ссылок для скорости работы
        try:
            html = requests.get(link).text
            html_list.append(html)
        except:
            continue
    return html_list

def get_linked_text(topic):
    html_list = get_linked_page(topic)
    words_list = []
    for html_content in html_list:
        words = re.findall("[а-яА-Я]{3,}", html_content)
        if len(words) > 0:
            words_list.append(words)
    return words_list

def get_linked_words(topic):
    words_list = get_linked_text(topic)
    rates_list = []
    for words in words_list:
        rate = {}
        for word in words:
            if word in rate:
                rate[word]+=1
            else:
                rate[word]=1
        rates = list(rate.items())
        rates.sort(key = lambda x: -x[1])
        rates_list.append(rates)
    return rates_list


In [9]:
dict1 = get_common_words('Россия')
pprint(dict1[:10])

[('России', 693),
 ('года', 242),
 ('Российской', 229),
 ('Федерации', 216),
 ('Россия', 211),
 ('отношения', 205),
 ('Российско', 162),
 ('году', 147),
 ('СССР', 128),
 ('страница', 101)]


In [10]:
dict2 = get_linked_words('Россия')
for item in dict2:
    print('*' * 42)
    pprint(item[:10])
    print('*' * 42)
    print('\n')

******************************************
[('России', 693),
 ('года', 242),
 ('Российской', 229),
 ('Федерации', 216),
 ('Россия', 211),
 ('отношения', 205),
 ('Российско', 162),
 ('году', 147),
 ('СССР', 128),
 ('страница', 101)]
******************************************


******************************************
[('карт', 8),
 ('всего', 4),
 ('карту', 3),
 ('категории', 3),
 ('Топокарты', 3),
 ('карты', 3),
 ('Выбор', 3),
 ('Нужную', 2),
 ('топографическую', 2),
 ('проще', 2)]
******************************************


******************************************
[('Служебная', 99),
 ('Обсуждение', 75),
 ('участника', 73),
 ('Участник', 63),
 ('Вклад', 55),
 ('обсуждение', 53),
 ('страница', 52),
 ('августа', 50),
 ('вклад', 50),
 ('отсутствует', 40)]
******************************************


